In [1]:
import sys
import os
import pandas as pd
import numpy as np

def topsis(input_file, weights, impacts, output_file):

    # ---------- File check ----------
    if not os.path.exists(input_file):
        raise FileNotFoundError("Input file not found")

    # ---------- Read Excel ----------
    df = pd.read_excel(input_file)

    if df.shape[1] < 3:
        raise ValueError("Input file must have at least 3 columns")

    # First column is alternative name
    data = df.iloc[:, 1:]

    # ---------- Numeric check ----------
    try:
        data = data.astype(float)
    except:
        raise ValueError("Criteria columns must be numeric")

    # ---------- Parse weights & impacts ----------
    weights = list(map(float, weights.split(",")))
    impacts = impacts.split(",")

    if len(weights) != data.shape[1]:
        raise ValueError("Number of weights must match number of criteria")

    if len(impacts) != data.shape[1]:
        raise ValueError("Number of impacts must match number of criteria")

    for i in impacts:
        if i not in ["+", "-"]:
            raise ValueError("Impacts must be + or - only")

    weights = np.array(weights)

    # ---------- Step 1: Normalize ----------
    norm = data / np.sqrt((data ** 2).sum())

    # ---------- Step 2: Weighted normalization ----------
    weighted = norm * weights

    # ---------- Step 3: Ideal best & worst ----------
    ideal_best = []
    ideal_worst = []

    for i in range(len(impacts)):
        if impacts[i] == "+":
            ideal_best.append(weighted.iloc[:, i].max())
            ideal_worst.append(weighted.iloc[:, i].min())
        else:
            ideal_best.append(weighted.iloc[:, i].min())
            ideal_worst.append(weighted.iloc[:, i].max())

    ideal_best = np.array(ideal_best)
    ideal_worst = np.array(ideal_worst)

    # ---------- Step 4: Distances ----------
    d_best = np.sqrt(((weighted - ideal_best) ** 2).sum(axis=1))
    d_worst = np.sqrt(((weighted - ideal_worst) ** 2).sum(axis=1))

    # ---------- Step 5: TOPSIS score ----------
    score = d_worst / (d_best + d_worst)

    df["Topsis Score"] = score
    df["Rank"] = score.rank(ascending=False).astype(int)

    # ---------- Save ----------
    df.to_excel(output_file, index=False)

    print("✅ TOPSIS calculation completed")
    print("📁 Output saved at:", output_file)



In [ ]:
if __name__ == "__main__":
    main()


In [2]:
topsis(
    "/content/data.xlsx",
    "1,1,1,1,1",
    "+,+,-,+,+",
    "/content/output.xlsx"
)


✅ TOPSIS calculation completed
📁 Output saved at: /content/output.xlsx
